In [ ]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier

from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode

import pickle

from nltk.tokenize import word_tokenize

In [ ]:
class VoteClassifier(ClassifierI):

  def __init__(self, *classifiers):
    self._classifiers = classifiers

  def classify(self, features):
    votes = []
    for c in self._classifiers:
      v = c.classify(features)
      votes.append(v)
    return mode(votes)

  def confidence(self, features):
    votes = []
    for c in self._classifiers:
      v = c.classify(features)
      votes.append(v)
    choice_votes = votes.count(mode(votes))
    conf = choice_votes / len(votes)
    return conf

In [ ]:
try:
    short_pos = open("positive.txt", "r", encoding="utf-8").read()
except UnicodeDecodeError:
    short_pos = open("positive.txt", "r", encoding="iso-8859-1").read()

try:
    short_neg = open("negative.txt", "r", encoding="utf-8").read()
except UnicodeDecodeError:
    short_neg = open("negative.txt", "r", encoding="iso-8859-1").read()

In [ ]:
documents = []

In [ ]:
allowed_word_types = ["J"]

In [ ]:
for p in short_pos.split('\n'):
  documents.append((p, "pos"))
  words = word_tokenize(p)
  pos = nltk.pos_tag(words)
  for w in pos:
    if w[1][0] in allowed_word_types:
      all_words.append(w[0].lower())

In [ ]:
for n in short_neg.split('\n'):
  documents.append((n, "neg"))
  words = word_tokenize(n)
  pos = nltk.pos_tag(words)
  for w in pos:
    if w[1][0] in allowed_word_types:
      all_words.append(w[0].lower())

In [ ]:
save_documents = open("pickled_algos/documents.pickle", "wb")
pickle.dump(documents, save_classifier)
save_documents.close()

In [ ]:
all_words = nltk.FreqDist(all_words)

In [ ]:
word_features = list(all_words.keys())[:5000]

In [ ]:
def find_features(document):
  words = word_tokenize(document)
  features = {}
  for w in word_features:
    features[w] = (w in words)

  return features

In [ ]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]

In [ ]:
random.shuffle(featuresets)
print(len(featuresets))

In [ ]:
training_set = featuresets[:10000]
testing_set = featuresets[10000:]

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
print(" NLTK-NB_classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

In [ ]:
save_classifier = open("pickled_algos/naivebayes.pickle", "wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

In [ ]:
classifier_f = open("naivebayes.pickle", "rb")
classifier_saved = pickle.load(classifier_f)
classifier_f.close()

In [ ]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print(" MNB_classifier accuracy percent:",(nltk.classify.accuracy(MNB_classifier, testing_set))*100)

In [ ]:
save_classifier = open("pickled_algos/MNB_classifier.pickle", "wb")
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

In [ ]:
BNB_classifier = SklearnClassifier(BernoulliNB())
BNB_classifier.train(training_set)
print(" BNB_classifier accuracy percent:",(nltk.classify.accuracy(BNB_classifier, testing_set))*100)

In [ ]:
save_classifier = open("pickled_algos/BNB_classifier.pickle", "wb")
pickle.dump(BNB_classifier, save_classifier)
save_classifier.close()

In [ ]:
LR_classifier = SklearnClassifier(LogisticRegression())
LR_classifier.train(training_set)
print(" LR_classifier accuracy percent:",(nltk.classify.accuracy(LR_classifier, testing_set))*100)

In [ ]:
save_classifier = open("pickled_algos/LR_classifier.pickle", "wb")
pickle.dump(LR_classifier, save_classifier)
save_classifier.close()

In [ ]:
SGD_classifier = SklearnClassifier(SGDClassifier())
SGD_classifier.train(training_set)
print(" SGD_classifier accuracy percent:",(nltk.classify.accuracy(SGD_classifier, testing_set))*100)

In [ ]:
save_classifier = open("pickled_algos/SGD_classifier.pickle", "wb")
pickle.dump(SGD_classifier, save_classifier)
save_classifier.close()

In [ ]:
LSV_classifier = SklearnClassifier(LinearSVC())
LSV_classifier.train(training_set)
print(" LSV_classifier accuracy percent:",(nltk.classify.accuracy(LSV_classifier, testing_set))*100)

In [ ]:
save_classifier = open("pickled_algos/LSV_classifier.pickle", "wb")
pickle.dump(LSV_classifier, save_classifier)
save_classifier.close()

In [ ]:
NSV_classifier = SklearnClassifier(NuSVC())
NSV_classifier.train(training_set)
print(" NSV_classifier accuracy percent:",(nltk.classify.accuracy(NSV_classifier, testing_set))*100)

In [ ]:
save_classifier = open("pickled_algos/NuSV_classifier.pickle", "wb")
pickle.dump(NuSV_classifier, save_classifier)
save_classifier.close()

In [ ]:
voted_classifier = VoteClassifier(classifier_saved,
                                  MNB_classifier,
                                  BNB_classifier,
                                  LR_classifier,
                                  SGD_classifier,
                                  LSV_classifier,
                                  NSV_classifier)

In [ ]:
print(" Voted_classifier accuracy percent:",(nltk.classify.accuracy(voted_classifier, testing_set))*100)

In [ ]:
def sentiment(text):
  feats = find_features(text)

  return voted_classifier.classify(feats), voted_classifier.confidence(feats)